In [ ]:
!pip install datasets

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import pandas as pd
import ast
import numpy as np
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import evaluate
import os
from evaluate import load
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import BertTokenizerFast, BertForTokenClassification, Trainer, TrainingArguments
from transformers import DistilBertTokenizerFast, DistilBertForTokenClassification
from transformers import BertTokenizer, BertForTokenClassification

In [ ]:
# Загрузка данных
df = pd.read_csv('eo.csv')

def safe_literal_eval(x):
    try:
        return ast.literal_eval(x)
    except (ValueError, SyntaxError):
        return np.nan  # или можно использовать пустой список: []

# Преобразование строк в списки
df['labeled'] = df['labeled'].apply(safe_literal_eval)

In [ ]:
def extract_tokens_labels(labeled):
    # Проверка, является ли labeled списком
    if isinstance(labeled, list):
        tokens = [token for token, _ in labeled]
        labels = [label for _, label in labeled]
    else:
        tokens, labels = [], []  # Пустые списки для NaN значений
    return tokens, labels

In [ ]:
df[['tokens', 'labels']] = df['labeled'].apply(lambda x: pd.Series(extract_tokens_labels(x)))

In [ ]:
# Создаем отображение меток в числовые значения
label_encoder = LabelEncoder()
all_labels = [label for sublist in df['labels'] for label in sublist]
label_encoder.fit(all_labels)

# Преобразуем метки в числовой формат
df['labels'] = df['labels'].apply(lambda x: label_encoder.transform(x).tolist())
# Преобразуем метки так, чтобы они начинались с 0, если это необходимо
df['labels'] = df['labels'].apply(lambda x: [label - 1 if label > 0 else label for label in x])

In [ ]:
# Создание Dataset
dataset = Dataset.from_pandas(df[['tokens', 'labels']])

In [ ]:
def prepare_data(df):
    tokens = df['tokens'].tolist()
    labels = df['labels'].tolist()
    return {'tokens': tokens, 'labels': labels}

In [ ]:
data = prepare_data(df)

if not data['tokens']:
    raise ValueError("No data available to split.")

# Разделение данных на обучающую и тестовую выборки
train_texts, test_texts, train_labels, test_labels = train_test_split(
    data['tokens'], data['labels'], test_size=0.3, random_state=42
)

In [ ]:
# Создание Dataset для обучения и тестирования
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

model_name = 'bert-base-multilingual-cased'
tokenizer = BertTokenizerFast.from_pretrained(model_name)
model = BertForTokenClassification.from_pretrained(model_name, num_labels=3)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples['tokens'],
        truncation=True,
        is_split_into_words=True,
        padding='max_length',
        return_offsets_mapping=True
    )

    labels = []
    for i, label_list in enumerate(examples['labels']):
        offset_mapping = tokenized_inputs.offset_mapping[i]
        label_ids = []
        previous_word_idx = None

        for j, (start, end) in enumerate(offset_mapping):
            if start == 0 and end == 0:
                label_ids.append(-100)  # Special tokens
            else:
                word_idx = tokenized_inputs.char_to_token(i, start, end - 1)
                if word_idx is None or word_idx >= len(label_list):
                    label_ids.append(-100)  # Token not mapped to a word or index out of range
                elif word_idx != previous_word_idx:
                    label_ids.append(label_list[word_idx])
                else:
                    label_ids.append(-100)  # Same word as previous token
                previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs['labels'] = labels
    return tokenized_inputs

In [ ]:
def remove_ignore_labels(labels):
    return [label for label in labels if label != -100]

In [ ]:
# Применение функции токенизации
train_dataset = train_dataset.map(tokenize_and_align_labels, batched=True)
val_dataset = val_dataset.map(tokenize_and_align_labels, batched=True)
test_dataset = test_dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/2946 [00:00<?, ? examples/s]

Map:   0%|          | 0/631 [00:00<?, ? examples/s]

Map:   0%|          | 0/632 [00:00<?, ? examples/s]

In [ ]:
train_dataset = train_dataset.remove_columns(['tokens'])
vsl_dataset = val_dataset.remove_columns(['tokens'])
test_dataset = test_dataset.remove_columns(['tokens'])

In [ ]:
accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

In [ ]:
def compute_metrics(pred):
    predictions = pred.predictions.argmax(axis=-1)  # Получаем индексы предсказанных меток

    # Сглаживаем predictions и labels
    flattened_predictions = [p for preds in predictions for p in preds]
    flattened_labels = [l for labels in pred.label_ids for l in labels]

    # Убираем -100 из предсказаний и меток
    valid_predictions = [p for p, l in zip(flattened_predictions, flattened_labels) if l != -100]
    valid_labels = [l for l in flattened_labels if l != -100]

    # Вычисление метрик
    accuracy = accuracy_metric.compute(predictions=valid_predictions, references=valid_labels)["accuracy"]
    precision = precision_metric.compute(predictions=valid_predictions, references=valid_labels, average="weighted")["precision"]
    recall = recall_metric.compute(predictions=valid_predictions, references=valid_labels, average="weighted")["recall"]
    f1 = f1_metric.compute(predictions=valid_predictions, references=valid_labels, average="weighted")["f1"]

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

In [ ]:
# Параметры обучения
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    load_best_model_at_end=True,
    report_to="none"  # Отключение W&B
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Инициализация Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# Тренировка модели
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.446374,0.888629,0.815241,0.888629,0.837411
2,0.457600,0.433816,0.886442,0.812121,0.886442,0.842895
3,0.390800,0.448671,0.889097,0.818567,0.889097,0.842419


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


TrainOutput(global_step=1107, training_loss=0.41681755569030704, metrics={'train_runtime': 1190.1881, 'train_samples_per_second': 7.426, 'train_steps_per_second': 0.93, 'total_flos': 2309361614641152.0, 'train_loss': 0.41681755569030704, 'epoch': 3.0})

In [ ]:
# Оценка финальных метрик на тестовом наборе
final_metrics = trainer.evaluate(test_dataset)
print("Итоговые метрики на тестовом наборе:", final_metrics)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Итоговые метрики на тестовом наборе: {'eval_loss': 0.4498614966869354, 'eval_accuracy': 0.8815151040851463, 'eval_precision': 0.807306214502757, 'eval_recall': 0.8815151040851463, 'eval_f1': 0.8366245508646941, 'eval_runtime': 18.3214, 'eval_samples_per_second': 34.495, 'eval_steps_per_second': 4.312, 'epoch': 3.0}


In [ ]:
# Сохранение обученной модели
model.save_pretrained('./train_model')
tokenizer.save_pretrained('./train_model')

('./train_model/tokenizer_config.json',
 './train_model/special_tokens_map.json',
 './train_model/vocab.txt',
 './train_model/added_tokens.json',
 './train_model/tokenizer.json')